# Quantum Geometric Machine Learning: Notebook

This notebook contains code for the QMLcontrolmodel class, the class that instantiates the various SubRiemannian, GRU RNN Greybox and Fully-connected Greybox models from the paper <i>Quantum Geometric Machine Learning for Quantum Circuits and Control </i> by Perrier et al (2020).

It also contains instructions on how to call the subRiemannian normal geodesic data generating simulation.py code to generate training (and test) data and how to extract control pulses and unitary estimates from each model.
It also includes example code for generalisation with respect to various test sets. It contains extensive comments to assist researchers. The code is built using TensorFlow 2.2 and Qutip > 4.0 among other packages.

Training times vary, with the FC Greybox model taking the longest on a Dell XPS 9850 i7 laptop (without using a GPU).

In [2]:

from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,optimizers,constraints,initializers,Model,backend
import pickle
import time
import zipfile    
import os
from itertools import combinations, accumulate
from operator import itemgetter
import qutip as qt
import scipy as sc
import random as rn
import operator
from functools import reduce
import csv

'''Optional for plotting using Qutip'''
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from matplotlib import animation
from IPython.display import HTML, Image
%matplotlib inline


from flattenunitary import flattenunitary
from flattenunitary import realise


'''simulation.py contains the code for generating subRiemannian geodesic training data'''
from simulation import GenData
'''customlayers.py contains the various customised greybox layers used in the models'''
from customlayers import *

## Data generation

Description:

GenData: a class from simulation.py that generates approximations to geodesics for use
as training data. 

Inputs are:

GenData(su2ndim, nseg, ntrain, time, holo, su2ngen, hscale)

with the following arguments:
* su2ndim: set n, used to set the dimension the Lie group SU($2^n$) from which unitaries are drawn;
* nseg: the number of segments of the approximate geodesic sequences $(U_j)$;
* ntrain: the number of training examples (number of approximate geodesic sequences);
* time: option in simulation where given number of segments nseg, can set
h = delta_t (the time-step for each unitary in the independent approximation
to Schrodinger's equation) to be time/nseg;
* holo: option to as to whether first unitary in loop generating $(U_j)$ is to be unitary generated
via the implementation of method in Boozer (2012) (used in Appendix to paper to compare 
subRiemannian method of generating geodesic approximations to the analytic method in Boozer (2012));
* su2ngen: Specify generators for initial condition $\Lambda_0$: 
    * 0 $: \Lambda_0 \in \Delta$ (the distribution) and \\
    * 1$: \Lambda_0 \in su(2^n)$ (full Lie algebra)
* hscale: Set $\Delta t_j$ (i.e. h in code or duration of pulse)

In [3]:
'''========================Example: generate and save data============================'''
'''Generate geodesic dataset by specifying hyperparameters and save as pickle file'''
'''Set dimension via n for SU(2^n).'''
nsu2dim = 1
'''Set the number of segements of the geodesic approximation'''
nseg = 5
'''Set the number of training examples'''
ntrain = 50
'''Set \delta tj (i.e. h in code or duration of pulse)'''
hscale = 0.1
'''Specify generators for initial condition \Lambda_0:
0: \Lambda_0 in \Delta (the distribution); 
1: \Lambda_0 \in su(2^n) (full Lie algebra)'''
su2gen = None #
'''Specify whether want to initialise unitary sequence using Boozer (2012) unitary, usually set to None'''
holo = None #whether training data to be generated from Boozer

'''Generate data'''
gentest = GenData(nsu2dim,nseg,ntrain,hscale,holo=holo, su2ngen=su2gen, hscale=hscale)

'''Convert data to form to save as pickle file'''
a = vars(gentest)

'''Note: using above variables allows automatic saving of name according to hyperparameters above'''
with open('su{0}_ntrain{1}_nseg{2}_hscale{3}_su2gen{4}.pickle'.format(
gentest.su2n_dim,gentest.ntraining, gentest.segments, gentest.h, gentest.su2ngen), 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)





## QMLcontrol model

Code below constructs a class QMLcontrolmodel to implement the three greybox quantum machine learning (QML) models that are described in the paper. Comments on functionality are in the code.

The class contains the following:
* Attributes such as the number of segments, training examples, distribution (control subalgebra generators) and so on
* Methods for preprocessing data
* Three primary models as methods: the SubRiemannian model, the Fully-connected (FC) Greybox model and the GRU RNN (LSTM) Greybox model
* methods for training and predicting using such models

In [54]:
class QMLcontrolmodel():
    """
    This class defines the machine learning models determining controls (generator coefficients).
    Arguments: 
    - gentest: the dataset class (see above) generated from simulation.py containing datasets in 
    various forms as attributes
    - genseg: set to 1
    """ 
    def __init__(self,gentest, genseg):
        '''Number of training examples'''
        self.ntraining = gentest.ntraining
        '''Convert identity in SU(2^n) to a tensor'''
        self.su2nidentity = tf.convert_to_tensor(gentest.su_dim_identity)
        '''Set dimension for SU(2^n)'''
        self.su2n_dim = gentest.su2n_dim
        '''Set dimension of realised SU(2^n) (i.e. dim(SU(2^n) * 2))'''
        self.su2n_realdim = gentest.su2n_dim * 2
        '''Set dimension of flattened realised unitary (for use in inputs to models below)'''
        self.flatrealdim = self.su2n_realdim ** 2
        '''Number of generators and coefficients used for prediction (legacy)'''
        self.genseg = genseg
        '''Set \Delta (distribution i.e. control subalgebra of Lie algebra) as attribute'''
        self.su_2n_delta = gentest.su_2n_delta
        '''Set as attribute hyperparameter determining if generators for \Lambda_0 drawn 
        from distribution or su(2^n)'''
        self.su2ngen = gentest.su2ngen
        '''Set dimension of control subalgebra (distribution \Delta)'''
        self.deltadim = np.asarray(gentest.su_2n_delta).shape[0]
        '''Set as attribute full Lie algebra su(2^n)'''
        self.su_2n = gentest.su_2n
        
        '''Conditional logic to ensure correct coefficients and generators for \Lambda_0 
        used for SubRiemannian model depending on su2ngen value (distribution or full su(2^n)) '''
        if self.su2ngen == 1:
            self.genlist = self.su_2n * self.genseg
            
            self.coef_numb = len(self.su_2n) * self.genseg
        else:
            self.genlist = self.su_2n_delta * self.genseg
            
            self.coef_numb = len(self.su_2n_delta) * self.genseg
        
        '''Set number of segments for approximate geodesic sequence (U)j'''
        self.segments = gentest.segments
        '''Set attribute number of segments x number of training examples, used in models below'''
        self.nseg = gentest.segments * gentest.ntraining
        '''self.tot is for tensor-based calculations where we have 
        segments x deltadim generators etc'''
        self.tot = self.deltadim * self.segments
        #self.coefzeros = np.asarray([0] * self.coef_numb)
        '''For main SubRiemannian model, set list for training history'''
        self.training_history = []
        '''For main SubRiemannian model, set list for validation history'''
        self.val_history = []
        '''Set vector of ones used for MSE with batch fidelity calculations'''
        self.fidones = gentest.fidones
        
        #self.fidones2 = np.asarray([1] * 3 * self.ntraining)
        '''Set vector of ones used for MSE with batch fidelity calculations in certain models'''
        self.fidones_seg = tf.ones((self.ntraining,self.segments))
        
        '''Set Ujarray (a list containing realised sequences (Uj)) as an array'''
        self.Ujarray = np.asarray(gentest.Uj_master_realised)
        '''Set Ujarray (a list containing sequences (Uj)) as an array'''
        self.Ujmasterarray = np.asarray(gentest.Uj_master_array)
        
        '''Set attribute that calls xtrainmethod(), a data preprocessing step for inputs into models.
        Specifically, the method takes as inputs (U_j) and outputs the final target unitary U_T
        by accumulating such that U_n @ ... @ U_0 = U_T'''
        self.xtrain = self.xtrainmethod()
        
        self.Ujm1 = np.asarray(gentest.Uj_master_realised).astype('float64')
        
        '''Set real and imaginary parts of (Uj) for inputs into models'''
        self.Ujseq_real = (tf.math.real(np.asarray(gentest.Uj_master_array)))
        self.Ujseq_imag = (tf.math.imag(np.asarray(gentest.Uj_master_array)))
        '''Set time-step h = \delta t'''
        self.h = gentest.h 
        '''Set attribute - number of coefficients equal to number of segments x number of generators'''
        self.coef_numb_seg = self.segments * tf.convert_to_tensor(gentest.su_2n_delta).shape[0]
        
        
        
        '''===Initialise models from methods'''
        '''SubRiemannian model'''
        self.mod1 = self.mod1()
        '''GRU RNN (LSTM) model'''
        self.mod_GRU_grey = self.LSTMGRUGrey()
        '''Fully-connected (FC) Greybox model'''
        self.modfc = self.modfc()
    
    
    '''===============Model: SubRiemannian model====================='''
    def mod1(self):
        '''Description: takes as inputs U_T, together with real and imaginary parts of the
        sequence of unitaries (U_j) as inputs and outputs estimated (\hat(U)_j) which are
        then compared with the true (U_j) using fidelity. This fidelity is then compared with
        a vector of ones (which are technically the labels in this model). The output of the model
        is a sequence of fidelities, one fidelity for each Uj for each training example. 
        Intermediate outputs, such as control pulses, Hamiltonian estimates etc are accessible via 
        constructing intermediate models in TensorFlow (see below)'''
        
        '''=====================Custom layer initialisation======================'''
        '''Custom layers are contained in customlayers.py'''
        
        '''===Hamiltonian estimation layers
        Description: Initialise a layer that takes controls for each generator in control subalgebra and
        multiplies them together for form an estimate of \Lambda_0
        (which is actually a Hamiltonian \hat{H}_j). Two layers
        are used for this purpose, one contains positive control values, the other negative.
        Two Hamiltonian estimates for \Lambda_0, a positive estimate \Lambda_0+ and negative \Lambda_0-
        are then constructed for each \Lambda_0, then added together (as are the controls).'''
        '''Positive layer'''
        ham_est_layer = ham_est(self.genlist,self.genseg, name='ham_layer')
        '''Negative layer'''
        ham_est_layer_neg = ham_est_neg(self.genlist,self.genseg, name='ham_layer_neg')
        
        '''========Fidelity layer
        Initialise fidelity layer (see customlayers.py for a description)'''
        fidelity_layer = fidelity(self.su2n_dim)
        
        
        '''========Unitary output and projection layer
        Description: layer recreates the method for generating approximations to normal 
        subRiemannian geodesics (\hat{U}_j) using the variational methods described in the paper drawn from
        Swaddle (2017). Inputs are \Lambda_0 and generators, outputs a sequence (\hat{U}_j).'''
        projev = identitylayer(self.su_2n_delta, self.su2nidentity, self.segments, self.h, name='Ujseq_est')
        
        '''Set dimensions for input'''
        origdim = (self.su2n_realdim,self.su2n_realdim)
        
        '''Set generic number of dense layer inputs (can be modified)'''
        intermediate_dim = 640
        
        '''Original inputs are the realised, flattened, target unitaries U_T. Note, these form the 
        training examples input into the model.'''
        original_inputs = tf.keras.Input(shape=origdim, name='flatinput')
        original_inputs1 = layers.Flatten()(original_inputs)
        
        '''Second inputs are the real and imaginary part of the target unitary,
        these are used to compare the estimated unitary against the target unitary.
        These are unflattened. The idea is to learn how to accurately input a flattened unitary
        constructed from delta generators using learnt weights (controls) and those generators.
        The targets are the same unitaries which are then used to calculate fidelities.'''
        targetreal = tf.keras.Input(shape=(self.su2n_dim,self.su2n_dim), name='target_real')
        targetimag = tf.keras.Input(shape=(self.su2n_dim,self.su2n_dim), name='target_imag')
        
        '''Clipped relu (customised)'''
        clip_relu = 'relu'
        '''=====================Dense layers======================
        Description: a fully-connected feed-forward intermediate neural network
        that takes as input U_T and outputs the control pulses for generating \Lambda_0 from
        its generating set.'''
        x = layers.Dense(intermediate_dim, activation=clip_relu)(original_inputs1)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        
        '''=====================Greybox layers======================'''
        '''Output positive controls'''
        output = layers.Dense(self.coef_numb,activation='tanh', name='controls')(x)
        '''Input positive controls to construct \Lambda_0+'''
        output1 = ham_est_layer(output)
        '''Output negative controls'''
        output2 = layers.Dense(self.coef_numb,activation='tanh', name='controls2')(x)
        '''Input negative controls to construct \Lambda_0-'''
        output2 = ham_est_layer_neg(output2)
        
        '''Reconstruct \Lambda_0 from \Lambda_0+ and \Lambda_0-'''
        Lambda0 = hamcomb(name='hamcomb1')([output1,output2])
        '''Input \Lambda_0 into projection/evolution layer to output (\hat{U}_j).
        Note: this layer outputs a list, first element is (\hat{U}_j). Second is controls i.e. x1[1]
        are the controls generated by the projection function used in the subRiemannian equations
        (see paper).'''
        x1 = projev(Lambda0)
        
        '''=====================Ground truth layer======================'''
        '''Input real and imaginary part of true (U_j) for batch fidelity layer'''
        inputUre = layers.Input(shape=(self.segments,self.su2n_dim,self.su2n_dim), name='input2', batch_size=None)
        inputUim = layers.Input(shape=(self.segments,self.su2n_dim,self.su2n_dim), name='input3', batch_size=None)
        
        '''=====================Output layer======================'''
        '''Input the real and imaginary part of true (U_j) and estimate (\hat{U}_j) into the fidelity function'''
        y = fidmulti_unitary(self.su2n_dim, name='finalfidelity')([inputUre,inputUim,x1[0]])
        
        '''Define model'''
        mod1 = tf.keras.Model(inputs=[original_inputs, inputUre, inputUim], outputs=y, name='mod1')
        '''Set optimizer (Adam)'''
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
        '''Compile, set loss as MSE'''
        mod1.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
        '''Print summary (for verification)'''
        mod1.summary()
        
        
        '''Return the SubRiemannian model'''
        return mod1
    
    
    '''===============Training: SubRiemannian model====================='''
    '''Description: trains SubRiemannian model using in-class data.
    For other data, the SubRiemannian model method or attribute can be called and 
    training of it can be done manually.'''
    def train_model_mod1(self, epochs):
        csv_logger = CSVLogger('train_mod1_su{0}_ntrain{1}_nseg{2}_hscale{3}_su2gen{4}.csv'
                       .format(self.su2n_dim,self.ntraining, self.segments, self.h, self.su2ngen))
        h = self.mod1.fit([self.xtrain,self.Ujseq_real,self.Ujseq_imag],self.fidones_seg, epochs=epochs, verbose=2, batch_size=10, validation_split = 0.25, shuffle=True, callbacks=[csv_logger])
        self.training_history = h.history["loss"]
        self.val_history       = h.history["val_loss"]
        
    '''===============Prediction: SubRiemannian model====================='''
    '''Description: generates predictions from trained model.'''
    def predict_mod1(self,inputs):
        return self.mod1.predict(inputs)
    
    
    '''===============SubModel: SubRiemannian model controls====================='''
    '''Description: outputs controls from trained model. Call after training.''' 
    '''Note: inputs needs to be [xtrain, Ujseq_real, Ujseq_imag] for a particular dataset.'''
    def controls_mod1(self,inputs):
        new_model = self.mod1
        controlmod = Model(inputs=new_model.input, outputs=new_model.get_layer('Ujseq_est').output[1])
        return controlmod.predict(inputs)
    
    
    '''===============SubModel: SubRiemannian model unitaries====================='''
    '''Description: outputs unitary estimates (\hat{U}_j) from trained model. Call after training.''' 
    '''Note: inputs needs to be [xtrain, Ujseq_real, Ujseq_imag] for a particular dataset.'''
    def unitaries_mod1(self,inputs):
        new_model = self.mod1
        unimod = Model(inputs=new_model.input, outputs=new_model.get_layer('Ujseq_est').output[0])
        return unimod.predict(inputs)
    
    def modgeneral(self,inputs):
        un = Model(inputs=self.mod1.input, outputs=self.mod1.outputs)
        un = un.predict(inputs)
        return un
    
    '''===============Model: Fully-connected (FC) Greybox model====================='''
    def modfc(self):
        
        '''Description: learns controls for sequence U = \prod_i exp(-\tau_i)
        where \tau_i are the elements of \Delta (with imaginary unit already
        included). See paper.'''
        
        '''=====================Custom layer initialisation======================'''
        '''Hamiltonian estimation layer, takes as inputs controls and generators, outputs
        estimated Hamiltonian sequence (\hat{H}_j).'''
        ham_est_layer_simple = ham_est_simple(self.su_2n_delta,self.segments, name='ham_layer')
        '''Quantum evolution layer, takes as input estimated Hamiltonian sequence (\hat{H}_j)
        and outputs estimated unitary sequence (\hat{U}_j).'''
        unitary_simp = unitary_simple(self.segments, self.su2nidentity, self.su2n_dim, name='unitaryoutput')
        
        '''See description in SubRiemannian layer for below'''
        origdim = (self.su2n_realdim,self.su2n_realdim)
        intermediate_dim = 640
        
        original_inputs = tf.keras.Input(shape=origdim, name='flatinput')
        original_inputs1 = layers.Flatten()(original_inputs)
        
        targetreal = tf.keras.Input(shape=(self.su2n_dim,self.su2n_dim), name='target_real')
        targetimag = tf.keras.Input(shape=(self.su2n_dim,self.su2n_dim), name='target_imag')
        clip_relu = 'relu'
        '''=====================Dense layers======================
        Description: a fully-connected feed-forward intermediate neural network
        that takes as input U_T and outputs the control pulses for generating 
        estimated Hamiltonian sequence (\hat{H}_j) using the control subalgebra (\Delta).'''
        x = layers.Dense(intermediate_dim, activation=clip_relu)(original_inputs1)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        x = layers.Dropout(0.2)(x)
        x = layers.Dense(intermediate_dim, activation=clip_relu)(x)
        
        '''=====================Greybox layers======================'''
        '''Output estimated control pulses'''
        output = layers.Dense(self.coef_numb_seg,activation='tanh', name='controls')(x)
        '''Combine the controls and generators from \Delta to construct 
        estimated Hamiltonian sequence (\hat{H}_j)'''
        output1 = ham_est_layer_simple(output)
        '''Quantum evolution layer that outputs estimated unitary sequence (\hat{U}_j)
        from estimated Hamiltonian sequence (\hat{H}_j)'''
        x1 = unitary_simp(output1)
        
        '''=====================Ground truth layer======================'''
        '''Input real and imaginary part of true (U_j) for batch fidelity layer'''
        inputUre = layers.Input(shape=(self.segments,self.su2n_dim,self.su2n_dim), name='input2', batch_size=None)
        inputUim = layers.Input(shape=(self.segments,self.su2n_dim,self.su2n_dim), name='input3', batch_size=None)
        
        '''=====================Output layer======================'''
        '''Input the real and imaginary part of true (U_j) and estimate (\hat{U}_j) into the fidelity function'''
        y = fidmulti_unitary(self.su2n_dim, name='finalfidelity')([inputUre,inputUim,x1])
        
        modfc = tf.keras.Model(inputs=[original_inputs, inputUre, inputUim], outputs=y, name='modfc')
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

        modfc.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
        
        print(modfc.summary())
        
        return modfc
    
    
    '''===============Training: FC Greybox model====================='''
    '''Description: trains FC Greybox model using in-class data.
    For other data, the FC Greybox model method or attribute can be called and 
    training of it can be done manually.'''
    def train_model_modfc(self, epochs):
        csv_logger = CSVLogger('train_modfc_su{0}_ntrain{1}_nseg{2}_hscale{3}_su2gen{4}.csv'
                       .format(self.su2n_dim,self.ntraining, self.segments, self.h, self.su2ngen))
        h = self.modfc.fit([self.xtrain,self.Ujseq_real,self.Ujseq_imag],self.fidones_seg, epochs=epochs, verbose=2, batch_size=10, validation_split = 0.25, shuffle=True, callbacks=[csv_logger])
        self.modfc_training_history = h.history["loss"]
        self.modfc_val_history       = h.history["val_loss"]
        
        

        
    '''===============Prediction: FC Greybox model====================='''
    '''Description: generates predictions from trained model.'''
    def predict_modfc(self,inputs):
        #self.LSTMGRUGrey().predict(inputs)
        return self.modfc.predict(inputs)
    
    '''===============SubModel: FC Greybox model controls====================='''
    '''Description: outputs controls from trained model. Call after training.''' 
    '''Note: inputs needs to be [xtrain, Ujseq_real, Ujseq_imag] for a particular dataset.'''
    def controls_modfc(self,inputs):
        new_model = self.modfc
        controlmod = Model(inputs=new_model.input, outputs=new_model.get_layer('controls').output)
        return controlmod.predict(inputs)
    
    '''===============SubModel: FC Greybox model unitaries====================='''
    '''Description: outputs unitary estimates (\hat{U}_j) from trained model. Call after training.''' 
    '''Note: inputs needs to be [xtrain, Ujseq_real, Ujseq_imag] for a particular dataset.'''
    def unitaries_modfc(self,inputs):
        new_model = self.modfc
        unimod = Model(inputs=new_model.input, outputs=new_model.get_layer('unitaryoutput').output)
        return unimod.predict(inputs)
    
    
    '''===============Model: GRU RNN (LSTM) Greybox model====================='''
    def LSTMGRUGrey(self):
        '''Description: learns controls c_i for sequence U = \prod_i exp(-c_i \tau_i)
        via a GRU RNN. See paper.'''
        
        '''=====================Custom layer initialisation======================'''
        origdim = (self.su2n_realdim,self.su2n_realdim)
        
        '''Hamiltonian estimation layer, takes as inputs controls and generators, outputs
        estimated Hamiltonian sequence (\hat{H}_j). In this case a Hamiltonian GRU layer
        uses the output of the GRU c^k to construct the Hamiltonian estimates using each generator \tau_k
        then the actual specific \hat{H}_j = \sum c^k \tau_k (imaginary unit incorporate in generators)'''
        ham1 = ham_est_GRU(self.su_2n_delta,self.segments, name='ham_GRU_layer')
        hsum = ham_sum_GRU(self.su_2n_delta,self.segments, name='ham_sum_GRU')
        
        inputUre = layers.Input(shape=(self.segments,self.su2n_dim,self.su2n_dim), name='GRU_input2', batch_size=None)
        inputUim = layers.Input(shape=(self.segments,self.su2n_dim,self.su2n_dim), name='GRU_input3', batch_size=None)
        original_inputs = tf.keras.Input(shape=origdim, name='GRU_flatinput')
        
        '''=====================GRU and Greybox layers======================'''
        '''Note: reshape -1 is to add None dimension so input dimensions for GRU are 
        batch, time, dimensions. Here each segment of (U_j) is associated with a time step,
        that is, the index j is effectively time t in the GRU.'''
        y = layers.Reshape((-1,self.flatrealdim,))(original_inputs)
        '''Input (U_T) into GRU to output estimated controls'''
        y = layers.GRU(self.tot, return_sequences=True, activation='tanh', name='GRUcoef')(y)
        '''Input controls into Hamiltonian estimation layer, one Hamiltonian per generator'''
        y = ham1(y)
        y = layers.Reshape((self.segments,self.deltadim,self.su2n_dim,self.su2n_dim))(y)
        '''Sum all the generator Hamiltonians to obtain \hat{H}_j = \sum c^k \tau_k and output
        a sequence (\hat{H}_j)'''
        y = hsum(y)
        '''Quantum evolution layer to generate unitary sequence (\hat{U}_j) from \hat{H}_j'''
        y = unitary_GRU(name='GRUunitary')(y)
        
        '''=====================Output layer======================'''
        '''Input the real and imaginary part of true (U_j) and estimate (\hat{U}_j) into the fidelity function'''
        y = fidmulti_unitary(self.su2n_dim, name='GRU_grey_fidelity')([inputUre,inputUim,y])
        gru_grey_mod = tf.keras.Model(inputs=[original_inputs, inputUre, inputUim], outputs=y, name='gru_grey_mod')
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

        gru_grey_mod.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
        
        print(gru_grey_mod.summary())
        return gru_grey_mod
    
    
    '''===============Method: preprocess unitary targets=====================
    Description: xtrainmethod takes as input the sequence of ground-truth unitaries
    (U_j), multiplies them together to obtain a corresponding sequence of target unitaries (U_T).'''
    def xtrainmethod(self):
        evolve = lambda U_j,U: U @ U_j
        acclist = []
        for list1 in self.Ujarray:
            acclist.append(list(accumulate(list1, evolve))[-1])
        xtrain = tf.convert_to_tensor(np.asarray(acclist))
        return xtrain

    
    '''===============Training: GRU RNN (LSTM) model====================='''
    '''Description: trains GRU RNN (LSTM) model using in-class data.
    For other data, the GRU RNN (LSTM) model method or attribute can be called and 
    training of it can be done manually.'''
    def train_model_LSTMGRUGrey(self, epochs):
        csv_logger = CSVLogger('train_GRUgrey_su{0}_ntrain{1}_nseg{2}_hscale{3}_su2gen{4}.csv'
                       .format(self.su2n_dim,self.ntraining, self.segments, self.h, self.su2ngen))
        '''Note: can modify batch_size as need be'''
        h = self.mod_GRU_grey.fit([self.xtrain,self.Ujseq_real,self.Ujseq_imag],self.fidones_seg, epochs=epochs, verbose=2, batch_size=None, validation_split = 0.25, shuffle=True, callbacks=[csv_logger])
        self.GRU_training_history = h.history["loss"]
        self.GRU_val_history = h.history["val_loss"]
        
    '''===============Prediction: GRU RNN (LSTM) model====================='''
    '''Description: outputs controls for predictions.'''
    '''Note: inputs needs to be [xtrain, Ujseq_real, Ujseq_imag] for a particular dataset.'''
    def predict_LSTMGRUGrey(self,inputs):
        #self.LSTMGRUGrey().predict(inputs)
        return self.mod_GRU_grey.predict(inputs)
        
    '''===============SubModel: GRU RNN (LSTM) model controls====================='''
    '''Description: outputs controls from trained model. Call after training.''' 
    '''Note: inputs needs to be [xtrain, Ujseq_real, Ujseq_imag] for a particular dataset.'''
    def controls_LSTMGRUGrey(self,inputs):
        new_model = self.mod_GRU_grey
        controlmod = Model(inputs=new_model.input, outputs=new_model.get_layer('GRUcoef').output)
        return controlmod.predict(inputs)
    
    '''===============SubModel: GRU RNN (LSTM) model unitaries====================='''
    '''Description: outputs unitary estimates (\hat{U}_j) from trained model. Call after training.''' 
    '''Note: inputs needs to be [xtrain, Ujseq_real, Ujseq_imag] for a particular dataset.'''
    def unitaries_GRU(self,inputs):
        new_model = self.mod_GRU_grey
        unimod = Model(inputs=new_model.input, outputs=new_model.get_layer('GRUunitary').output)
        return unimod.predict(inputs)
    
    
    '''===============Model: Simple GRU model====================='''
    '''Note: not used in paper but was one of the models trialled. May be of use for researchers.'''
    def LSTMsimple(self,inputs):
        '''Description: generates estimated sequence of subunitaries (U_j) from
        input U_T using only variable tensors (not generators) using GRU LSTM.
        Aim is global decomposition of U_T into subunitaries. It in effect is a greybox version
        of a GRU RNN global decomposition algorithm (finding (U_j) from (U_T)) as distinct
        from the main GRU RNN (LSTM) greybox model above which uses the GRU layer to estimate
        the control pulses as part of the greybox approach.'''
        origdim = (self.segments,self.su2n_realdim,self.su2n_realdim)
        totalflat = self.segments*self.su2n_dim*self.su2n_dim
        original_inputs = tf.keras.Input(shape=origdim, name='Ujseqinput', batch_size=None)
        y1 = layers.Reshape((-1,self.segments * self.su2n_realdim * self.su2n_realdim,))(original_inputs)
        GRUreal = layers.GRU(totalflat, return_sequences=True, activation='tanh', name='GRUreal')(y1)
        GRUimag = layers.GRU(totalflat, return_sequences=True, activation='tanh', name='GRUimag')(y1)
        Reshapereal = layers.Reshape((self.segments,self.su2n_dim,self.su2n_dim,))(GRUreal)
        Reshapeimag = layers.Reshape((self.segments,self.su2n_dim,self.su2n_dim,))(GRUimag)
        inputUre = layers.Input(shape=(self.segments,self.su2n_dim,self.su2n_dim,), name='input2', batch_size=None)
        inputUim = layers.Input(shape=(self.segments,self.su2n_dim,self.su2n_dim,), name='input3', batch_size=None)
        y = recombcomplex(name='unitaryoutput')([Reshapereal,Reshapeimag])
        y_out = fidmulti_unitary(gentest._su2n_dim, name='finalfidelity')([inputUre,inputUim,y])
        LSTMsimpmod = Model(inputs=[original_inputs, inputUre, inputUim], outputs=y_out, name='LSTMsimple')
        optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
        MSE = tf.keras.losses.MeanSquaredError()
        LSTMsimpmod.compile(optimizer, loss=MSE)
        print(LSTMsimpmod.summary())
        
        return LSTMsimpmod
    
    
    '''===============Method: flatten unitary=====================
    Description: Method takes target unitary in form of numpy arrayas input, flattens and also calculates 
        real and imaginary parts. Output is a 3-tuple of flattened (realised) unitary, 
        real part of operator and imaginary part of unitary.'''
    def flatun(self, unitary):
        Ujm_re = np.real(unitary)
        Ujm_im = np.imag(unitary)
        Ujm_top = np.concatenate((Ujm_re, -Ujm_im), axis=1)
        Ujm_bottom = np.concatenate((Ujm_im, Ujm_re), axis=1)
        Ujm = np.vstack((Ujm_top, Ujm_bottom)).astype('float64')
        Ujm = Ujm.flatten()
        Ujm = np.expand_dims(Ujm,0)
        Ujm_re = np.expand_dims(Ujm_re,0)
        Ujm_im = np.expand_dims(Ujm_im,0)
        return [Ujm,Ujm_re,Ujm_im]
    


## Worked example

In the example below, we demonstrate how to load saved datasets (see above and simulation.py for a description of dataset generation), training and generalisation.

### Load datasets

Datasets are saved as pickle files. They are generated via simulation.py which provides code that simulates methods of generating discretised approximations to normal subRiemannian geodesics. 

The simulation.py code creates a class with attributes and methods specified by certain hyperparameters (such as n in SU$(2^n)$). That class data is saved using pickle.

To load pickle files in a way that enables access to attributes and methods as if the data were generated directly by the simulation.py class, we need to convert the pickle input using the
namedtuple method below. 

The example below loads a dataset with the following key parameters:
* Lie group SU$(2)$
* segments: 10
* training examples: 1000
* $\Lambda_0$ is drawn from $\frak{su}(2)$
* $h = 0.1$


In [9]:
'''========================Example: load saved data============================'''

'''Note: here need to specify su2dim = 2^n, not just n'''
su2dim = 2
nseg = 10
ntrain = 1000
hscale = 0.1 #\delta tj (i.e. h in code or duration of pulse)
su2ngen = 1 #0: \Lambda_0 in \Delta; 1: \Lambda_0 \in su2n
holo = None #whether training data to be generated from Boozer

with open('su{0}_ntrain{1}_nseg{2}_hscale{3}_su2gen{4}.pickle'
          .format(su2dim,ntrain, nseg, hscale, su2ngen), 'rb') as handle:
    gentest = pickle.load(handle)

'''Convert dictionary to tuple so behaves like an object once loaded'''
from collections import namedtuple
'''Note: need to save as namedtuple to recreate class attributes/methods for input into
code if loading from pickle'''
gentest = namedtuple('Struct', gentest.keys())(*gentest.values())

### Initialise QMLcontrol model

Here we initialise the QMLcontrolmodel

In [55]:
genmod = QMLcontrolmodel(gentest,1)
'''Note: set the second argument, genseg, to 1'''


Model: "mod1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
flatinput (InputLayer)          [(None, 4, 4)]       0                                            
__________________________________________________________________________________________________
flatten_20 (Flatten)            (None, 16)           0           flatinput[0][0]                  
__________________________________________________________________________________________________
dense_140 (Dense)               (None, 640)          10880       flatten_20[0][0]                 
__________________________________________________________________________________________________
dropout_120 (Dropout)           (None, 640)          0           dense_140[0][0]                  
_______________________________________________________________________________________________

Model: "modfc"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
flatinput (InputLayer)          [(None, 4, 4)]       0                                            
__________________________________________________________________________________________________
flatten_21 (Flatten)            (None, 16)           0           flatinput[0][0]                  
__________________________________________________________________________________________________
dense_147 (Dense)               (None, 640)          10880       flatten_21[0][0]                 
__________________________________________________________________________________________________
dropout_126 (Dropout)           (None, 640)          0           dense_147[0][0]                  
______________________________________________________________________________________________

'Note: set the second argument, genseg, to 1'

### Train model

Here we show an example of training the SubRiemannian model on the pickle data loaded above.

Doing so will output data into both history attributes in the class and also CSVs via the callback in the model code (using CSVLogger). Note: verbose is set to 2 in the class above, this can be modified.

In [56]:
'''Select epochs - generally between 200-500 is sufficient'''
epochs = 2

'''Select model (comment out unused models). Note that:
- mod1 is the SubRiemannian modeltribute as an attribute
- mod_GRU_grey is the GRU RNN Greybox model as an attribute
- modfc is the FC Greybox model as an attribute'''
#genmod.train_model_mod1(epochs)
genmod.train_model_LSTMGRUGrey(epochs)
# genmod.train_model_modfc(epochs)

Epoch 1/2
24/24 - 1s - loss: 5.9419e-04 - val_loss: 3.3978e-04
Epoch 2/2
24/24 - 0s - loss: 2.4539e-04 - val_loss: 1.6527e-04


### Extract controls

To extract controls, we call the control submodels for the respective model we have selected.
This should be done <i>after</i> training the model in question. The controls are those that can be applied to the generators to recreate $(\hat{U}_j)$. 

In particular, the GenData class will contain an attribute genmod.su_2n_delta which is the distribution (control subalgebra) in question. The order of the distribution is fixed e.g. for SU(2) it may be X,Y.

The controls output a single sequence. So if there are two generators and ten segements, then the control output will be a vector of 20 elements. The first two are the controls for generators X,Y respectively for generating the estimated Hamiltonian $\hat{H}_1$, the third and fourth elements are the two controls for X and Y to generate $\hat{H}_2$ and so on. 

The estimated unitaries $\hat{U)_j$ are then obtained via application of the time-independent approximation to Schrodinger's equation i.e:

$$\hat{U}_j = \exp(-h \hat{H}_j)$$

where the imaginary unit $i$ is already included in $\hat{H}_j$ because it is incorporated into the distribution (see paper).

In [59]:
'''Controls - '''
genmod.controls_LSTMGRUGrey([genmod.xtrain, genmod.Ujseq_real,genmod.Ujseq_imag])
# genmod.controls_modfc([genmod.xtrain, genmod.Ujseq_real,genmod.Ujseq_imag])
#genmod.controls_mod1([genmod.xtrain, genmod.Ujseq_real,genmod.Ujseq_imag])

(1000, 1, 20)

### Extract unitaries

To extract unitary estimates $(\hat{U}_j)$, we call the unitary submodels for the respective model we have selected.
This should be done <i>after</i> training the model in question.

In [58]:
genmod.unitaries_mod1([genmod.xtrain, genmod.Ujseq_real,genmod.Ujseq_imag])
# genmod.unitaries_modfc([genmod.xtrain, genmod.Ujseq_real,genmod.Ujseq_imag])
# genmod.unitaries_GRU([genmod.xtrain, genmod.Ujseq_real,genmod.Ujseq_imag])

array([[[[ 9.99994023e-01-3.45731721e-03j,
          -9.53507068e-06-3.29659610e-08j],
         [ 9.53507068e-06-3.29659610e-08j,
           9.99994023e-01+3.45731721e-03j]],

        [[ 9.99899461e-01-1.37292660e-02j,
          -3.54587342e-03-4.86871345e-05j],
         [ 3.54587342e-03-4.86871345e-05j,
           9.99899461e-01+1.37292660e-02j]],

        [[ 9.99955699e-01+7.66408257e-03j,
           5.46435570e-03-4.18811286e-05j],
         [-5.46435570e-03-4.18811286e-05j,
           9.99955699e-01-7.66408257e-03j]],

        ...,

        [[ 9.99975149e-01+2.56536523e-03j,
           6.56654586e-03-1.68460070e-05j],
         [-6.56654586e-03-1.68460070e-05j,
           9.99975149e-01-2.56536523e-03j]],

        [[ 9.99972219e-01+6.19963551e-03j,
           4.13829586e-03-2.56566387e-05j],
         [-4.13829586e-03-2.56566387e-05j,
           9.99972219e-01-6.19963551e-03j]],

        [[ 9.99992442e-01-2.25826381e-03j,
          -3.16480075e-03-7.14700901e-06j],
         [ 3.164800

### Saving models

In [13]:
'''Save model'''
genmod.mod1.save('SubR')
'''Generate predictions to output a list of fidelities for each U_j for each training example.'''
mod_pred =  genmod.mod1.predict([genmod.xtrain, genmod.Ujseq_real,genmod.Ujseq_imag])
'''Save the predictions'''
np.savetxt("SubR_fidelity.csv", mod_pred, delimiter=",")
print('Save completed.')

INFO:tensorflow:Assets written to: SubR\assets


INFO:tensorflow:Assets written to: SubR\assets


### Access intermediate layers of model

The code below demonstrates how to construct submodels to access intermediate outputs of the model. For controls and unitary sequences, this is hard-coded into the QMLcontrolmodel class (see submodel sections).

This is necessary because the sequences of control pulses or estimated Hamiltonians or unitaries are themselves only intermediate outputs of the models.

In the case of the SubRiemannian model below, the layer projev outputs a list of length two:
* the first entry is an array (tensor) of the sequence of estimated unitaries
* the second is a list of the controls generated via the trace of the projection function

In [30]:
'''===========SubRiemannian: Load coefficient and unitary sub-models=============='''
'''Note: is for main model mod1 in QML model class'''
'''Load a pretrained model'''
new_model = tf.keras.models.load_model('SubR')

'''===========Unitary submodel: extracts output from unitary output layer==========='''
'''Description: create a submodel which takes the same inputs but outputs the layer which
undertakes the quantum evolution of Hamiltonians i.e. the layer that outputs the estimated unitaries.
Here use output index (0)'''
unimod = Model(inputs=new_model.input, outputs=new_model.get_layer('Ujseq_est').output[0])
unimodpred = unimod.predict([genmod.xtrain, genmod.Ujseq_real,genmod.Ujseq_imag])
unimodpred

#np.savetxt("SubRoutput.csv", unimodpred, delimiter=",")

'''===========Control pulse submodel: extracts control pulses==========='''
'''Description: extracts controls that are generated in the SubRiemannian model (index 1)'''
controlmod = Model(inputs=new_model.input, outputs=new_model.get_layer('Ujseq_est').output[1])
controlpred = controlmod.predict([genmod.xtrain, genmod.Ujseq_real,genmod.Ujseq_imag])
#np.savetxt("SubRcontrols.csv", controlpred, delimiter=",")

### Testing/generalisation against estimates

Below we provide an example of different methods of generalisation of the models by using the models to predict using out-of-sample datasets. There are a variety of ways of doing this, including by reinitialising the GenData geodesic generating code and inputting the unitary targets etc from that new class of data, or by randomly generating unitary targets $U_T$, generating sequences $(\hat{U}_j)$ and then comparing how well $\hat{U}_T = \hat{U}_n...\hat{U}_0$ approximates $U_T$, or by generating $U_T$ as rotations about the $z$-axis by a random angle, $\theta$.

In [32]:
'''=================Generalisation==================='''
'''Step 1: select hyperparameters for datsets (load existing or generate new ones)'''
su2dim = 2
nseg = 10
ntrain = 1000
hscale = 0.1 #\delta tj (i.e. h in code or duration of pulse)
su2ngen = None #0: \Lambda_0 in \Delta; 1: \Lambda_0 \in su2n
holo = None #whether training data to be generated from Boozer

'''Load pre-saved model'''
modstring = 'SubR'

'''Option: random generation of z-rotation unitaries as test set of 
target unitaries U_T. Set to 1 to do so.'''
testz = 1 #1

'''==========Load dataset========='''
'''Note: even if random unitaries or some other test set is being used, the prediction for
out of sample code will still require input of the ground-truth real and imaginary parts of (U_j)
though these are not used in prediction, they must be input because of how the models are constructed.'''
with open('su{0}_ntrain{1}_nseg{2}_hscale{3}_su2gen{4}.pickle'
          .format(su2dim,ntrain, nseg, hscale, su2ngen), 'rb') as handle:
    gentest = pickle.load(handle)

'''Convert dictionary to tuple so behaves like an object once loaded'''
from collections import namedtuple
gentest = namedtuple('Struct', gentest.keys())(*gentest.values())

genmod = QMLcontrolmodel(gentest,1)

'''============FINAL TESTING/GENERALISATION CODE==============='''

'''===========SubRiemannian: Load coefficient and unitary sub-models=============='''
'''Note: is for main model mod1 in QML model class'''

'''Load the existing model'''
new_model = tf.keras.models.load_model(modstring)

'''Create a submodel to extract unitaries'''
unimod = Model(inputs=new_model.input, outputs=new_model.get_layer('Ujseq_est').output)

'''=============Testing against random unitaries==========='''

pi = np.pi
'''=============Function: realise unitaries'''
'''Description: converts complex unitaries into realised form for input into model'''
def realise1(U1):
    U1_re = U1.real
    U1_im = U1.imag
    '''Create realisation of U as: [[X, -Y],[Y,X]]'''
    complex_U1_top = np.concatenate((U1_re, -U1_im), axis=1)
    complex_U1_bottom = np.concatenate((U1_im, U1_re), axis=1)
    '''Stack them both together'''
    complex_U1 = np.vstack((complex_U1_top, complex_U1_bottom))
    return complex_U1

'''=============Method: looping to generate random unitaries============='''
'''Description: generates random unitaries for test set via loop,
here we generate random angle to rotate about z-axis by. The stack lists are to preprocess
them into a form so that they can be input as target unitaries U_T in the models.'''
realised_U1stack = []
complex_U1stack = []
'''List to save random angles'''
anglelist = []
for i in range(gentest.ntraining):
    if testz == 1:
        rnangle = np.random.uniform(-2*pi,2*pi)
        anglelist.append(rnangle)
        U1 = qt.rz(rnangle).data.toarray()
    else:
        U1 = qt.rand_unitary(gentest.su2n_dim).data.toarray()
    
    
    complex_U1stack.append(U1)
    realised_U1 = realise1(U1)
    realised_U1stack.append(realised_U1)
    
realisedU1array = np.asarray(realised_U1stack)
complexU1array = np.asarray(complex_U1stack)
#testfid = new_model.predict([realisedU1array, genmod.Ujseq_real,genmod.Ujseq_imag])
testunitary = unimod.predict([realisedU1array, genmod.Ujseq_real,genmod.Ujseq_imag])





'''=============Function: xtrainmethod============='''
'''Description: replicates xtrainmethod from QMLcontrolmodel class above, that is
it creates U_T by multiplying each U_j in a sequence together (cumulant)'''
from itertools import accumulate
Ujarray = np.asarray(testunitary[0])[::-1]
def xtrainmethod(Ujarray):
    evolve = lambda U_j,U: U @ U_j
    acclist = []
    for list1 in Ujarray:
        acclist.append(list(accumulate(list1, evolve))[-1])
    xtrain = tf.convert_to_tensor(np.asarray(acclist))
    return xtrain
xtrain_est = xtrainmethod(Ujarray)
xtrain_est.shape

'''Convert to tensor for input into models'''
complexU1tensor = tf.convert_to_tensor(complexU1array)


'''=============Function: fidelity function============='''
'''Description: estimates fidelities of the true U_T and the estimate \hat{U}_T'''
def fidelityfunction(UTtrue,UTestimate):
    fid = tf.square(tf.abs(tf.linalg.trace(tf.matmul(UTtrue,UTestimate, adjoint_b=True))))/(UTtrue.shape[1]**2)
    return fid

'''=============Function: bulk fidelity function============='''
'''Description: estimates fidelities of the true U_T and the estimate \hat{U}_T but en mass
(in bulk) to avoid loops (faster implementation using tensors rather than loops)'''
def fidelitybulk(UTtrue,UTestimate):
    A1 = tf.einsum('aijk,aikm->aijm', UTtrue,tf.transpose(UTestimate,perm=[0,1,3,2],conjugate=True))
    fid = tf.square(tf.abs(tf.linalg.trace(A1)))/(UTtrue.shape[2]**2)
    return fid

'''=============Method: fidelity list============='''
'''Description: calculates fidelity of estimated UT v true UT and inputs into list named fidlist.
- complexU1tensor: the (true) randomly generated UT targets.
- xtrain_est: the estimates of UT i.e. UThat generated by the applicable model.
- fidelityfunction: comparison of fidelity F(UT,UThat). Note the estimate is second.
'''
fidlist = (fidelityfunction(complexU1tensor, xtrain_est))

'''=============Method: fidelity comparison============='''
'''Description: compares each random (true) UT with UTtrain (UT from the training data)
to estimate how similar UT is to the class UTtrain. The way this code part works below
is described in the paper. In short, it takes a particular target U_T. We have above
the means of assessing the fidelity of U_T versus the estimate \hat{U}_T i.e.
F(U_T,\hat{U}_T). But we want to assess the extent to which higher F(U_T,\hat{U}_T) is as a result
of the randomly generated U_T being more similar to the training set upon which the model was trained.
To do this, we calculate the fidelity of U_T with each U_T in the training set and average it.
This gives us an indication of how similar U_T was to the U_T from the training set. See paper for
a discussion.'''

xtrain_orig = xtrainmethod(genmod.Ujmasterarray)
xtrainstack = tf.stack([xtrain_orig] * xtrain_orig.shape[0], axis=0)
Ustack = tf.stack([complexU1tensor] * complexU1tensor.shape[0],axis=1)

fbulk = fidelitybulk(Ustack,xtrainstack)

'''Bulk average operator fidelity i.e. comparing each estimated UT 
to all UTs in xtrain (random target UTs)'''
fbulkav = tf.math.reduce_mean(fbulk, axis=1)

fidconcat = tf.transpose(tf.stack([fidlist,fbulkav], axis=0, name='concat'),perm=[1,0])
fidarray = fidconcat.numpy()
agmx = tf.argmax(fidconcat[:,0]).numpy()
A = fidarray[fidarray[:,1].argsort()]

'''Another copy to save but columns switched'''
angletens = tf.cast(tf.transpose(tf.convert_to_tensor(anglelist)), dtype=tf.float64)


'''Note: in the case of test set being randomly generated z-rotation unitaries.'''
if testz == 1:
    fidconcat1 = tf.transpose(tf.stack([fbulkav,fidlist,angletens], axis=0))
    
    fidarray1 = fidconcat1.numpy()
    A1 = fidarray1[fidarray1[:,1].argsort()]
    np.savetxt('modangle{0}_angle_su{1}_ntrain{2}_nseg{3}_hscale{4}_su2gen{5}_z{6}.csv'
              .format(modstring,su2dim,ntrain, nseg, hscale, su2ngen,testz), A1, delimiter=",")


fidconcat1 = tf.transpose(tf.stack([fbulkav,fidlist], axis=0, name='concat'),perm=[1,0])

fidarray1 = fidconcat1.numpy()

A1 = fidarray1[fidarray1[:,1].argsort()]

np.savetxt('mod{0}_su{1}_ntrain{2}_nseg{3}_hscale{4}_su2gen{5}_z{6}.csv'
              .format(modstring,su2dim,ntrain, nseg, hscale, su2ngen,testz), A1, delimiter=",")


'''Correlation analysis to determine how significant the similarity of U_T to those in the test set is.'''
from scipy.stats.stats import pearsonr, spearmanr  
a = A[:,0]
b = A[:,1]   
spearman = spearmanr(a,b)
pearson = pearsonr(a,b)
corrmx = np.corrcoef(a,b)
print(spearman,pearson,corrmx)
np.savetxt('modcorr{0}_su{1}_ntrain{2}_nseg{3}_hscale{4}_su2gen{5}.csv'
              .format(modstring,su2dim,ntrain, nseg, hscale, su2ngen), [spearman,pearson], delimiter=",")

Model: "mod1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
flatinput (InputLayer)          [(None, 4, 4)]       0                                            
__________________________________________________________________________________________________
flatten_6 (Flatten)             (None, 16)           0           flatinput[0][0]                  
__________________________________________________________________________________________________
dense_42 (Dense)                (None, 640)          10880       flatten_6[0][0]                  
__________________________________________________________________________________________________
dropout_36 (Dropout)            (None, 640)          0           dense_42[0][0]                   
_______________________________________________________________________________________________

Model: "modfc"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
flatinput (InputLayer)          [(None, 4, 4)]       0                                            
__________________________________________________________________________________________________
flatten_7 (Flatten)             (None, 16)           0           flatinput[0][0]                  
__________________________________________________________________________________________________
dense_49 (Dense)                (None, 640)          10880       flatten_7[0][0]                  
__________________________________________________________________________________________________
dropout_42 (Dropout)            (None, 640)          0           dense_49[0][0]                   
______________________________________________________________________________________________

C:\Users\elija\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: DeprecationWarning: Importing functions/classes of the qip submodule directly from the namespace qutip is deprecated. Please import them from the submodule instead, e.g.
from qutip.qip.operations import cnot
from qutip.qip.circuit import QubitCircuit



SpearmanrResult(correlation=0.02883754083754084, pvalue=0.3623101241284554) (0.028418322332049557, 0.36933293101126446) [[1.         0.02841832]
 [0.02841832 1.        ]]
